In [1]:
# Import all libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import Recommenders as Recommenders
import Evaluation as Evaluation

In [2]:
df1 = pd.read_csv("ratings_Electronics.csv") # Read file

In [3]:
df1.shape

(7824481, 4)

In [4]:
df1.columns = ['UserId', 'ProductID', 'Ratings', 'TimeStamp'] # Add names to coulmns of the dataset
df1.head()

,UserId,ProductID,Ratings,TimeStamp
0,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
1,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
2,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
3,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
4,A1QGNMC6O1VW39,0511189877,5.0,1397433600


In [5]:
df1 = df1.drop('TimeStamp', axis = 1) # REmove column "TimeStamp" 

In [6]:
# Linear encoding for 'UserId' and 'ProductID' columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df1_le = df1
#df1_le['UserId'] = le.fit_transform(df1_le['UserId'])
df1_le['ProductID'] = le.fit_transform(df1_le['ProductID'])
df1_le.head()

,UserId,ProductID,Ratings
0,A2CX7LUOHB2NDG,0,5.0
1,A2NWSAGRHCP8N5,1,1.0
2,A2WNBOD3WNDNKT,1,3.0
3,A1GI0U4ZRJA8WN,1,1.0
4,A1QGNMC6O1VW39,2,5.0


In [7]:
df1_user = df1
df1_user['ProductID'] = le.fit_transform(df1_user['ProductID'])
df1_user.head()

,UserId,ProductID,Ratings
0,A2CX7LUOHB2NDG,0,5.0
1,A2NWSAGRHCP8N5,1,1.0
2,A2WNBOD3WNDNKT,1,3.0
3,A1GI0U4ZRJA8WN,1,1.0
4,A1QGNMC6O1VW39,2,5.0


In [8]:
# Check the user-wise rating and product-wise rating
df2 = df1_le.groupby(['UserId']).agg({'Ratings': 'count'}).reset_index()
users = df2.UserId.unique()
df2_items = df1_le.groupby(['ProductID']).agg({'Ratings': 'count'}).reset_index()

In [9]:
len(users)

4201696

In [10]:
items = df2_items.ProductID.unique()
len(items)

476001

In [11]:
#Considering the products and user with ratings > 100 for simplicity
df3 = df2[df2['Ratings']>100]
df3_items = df2_items[df2_items['Ratings']>100]

In [12]:
df3.shape

(280, 2)

In [13]:
df3_items.shape

(13077, 2)

In [14]:
b = df3.sort_values(['Ratings','UserId'],ascending=[0, 1]).reset_index() # dumy variables a and b

In [15]:
a = df3_items.sort_values(['Ratings','ProductID'],ascending=[0, 1]).reset_index()

In [16]:
grouped_sum_items = df3_items['Ratings'].sum()
df3_items['Percentage']  = df3_items['Ratings'].div(grouped_sum_items)*100
df4_items = df3_items.sort_values(['Ratings', 'ProductID'], ascending = [0,1]).reset_index()

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [18]:
grouped_sum = df3['Ratings'].sum()
df3['Percentage']  = df3['Ratings'].div(grouped_sum)*100
df4 = df3.sort_values(['Ratings', 'UserId'], ascending = [0,1]).reset_index()

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [19]:
df4.head()

,index,UserId,Ratings,Percentage
0,3263531,A5JLAU2ARJ0BO,520,1.200674
1,3512451,ADLVFFE4VBT8,501,1.156803
2,2989526,A3OXHLG6DIBRW8,498,1.149876
3,3291008,A6FIAB28IS79,431,0.995174
4,3284634,A680RUE1FDO8B,406,0.937449


In [20]:
df4_items.head()

,index,ProductID,Ratings,Percentage
0,308397,308397,18244,0.409143
1,429571,429571,16454,0.369000
2,327307,327307,14172,0.317823
3,102803,102803,12285,0.275505
4,296624,296624,12226,0.274182


# Building Popularity based Recommender model

In [21]:
# Split data 
X, y = train_test_split(df4_items, test_size = 0.3, random_state = 24)
X.head()

,index,ProductID,Ratings,Percentage
6387,358651,358651,193,0.004328
2679,110969,110969,384,0.008612
12600,226422,226422,104,0.002332
4172,205391,205391,275,0.006167
4542,96763,96763,255,0.005719


In [22]:
train_data_grouped  = X.sort_values(['Ratings', 'ProductID'], ascending = [0,1]).reset_index()
max(train_data_grouped.Ratings)

16454

In [23]:
#Generate a recommendation rank based based on Rating 
train_data_grouped['Rank'] = train_data_grouped['Ratings'].rank(ascending=0, method='first') 
          
#Get the top 5 recommendations 
popularity_recommendations = train_data_grouped.head(5) 
popularity_recommendations 

,level_0,index,ProductID,Ratings,Percentage,Rank
0,1,429571,429571,16454,0.369000,1.0
1,2,327307,327307,14172,0.317823,2.0
2,3,102803,102803,12285,0.275505,3.0
3,4,296624,296624,12226,0.274182,4.0
4,6,178812,178812,10276,0.230451,5.0


In [24]:
def recommend(user_id):     
    user_recommendations = popularity_recommendations 
          
    user_recommendations['userID'] = user_id 
     
    cols = user_recommendations.columns.tolist() 
    cols = cols[-1:] + cols[:-1] 
    user_recommendations = user_recommendations[cols] 
          
    return user_recommendations 

In [25]:
# Find recommendation for different userIDs
find_recom = [5000, 243, 5, 34532, 99999]
for i in find_recom:
    print("Here is the recommendation for the userId: %d\n" %(i))
    print(recommend(i))    
    print("\n") 

Here is the recommendation for the userId: 5000

   userID  level_0   index  ProductID  Ratings  Percentage  Rank
0    5000        1  429571     429571    16454    0.369000   1.0
1    5000        2  327307     327307    14172    0.317823   2.0
2    5000        3  102803     102803    12285    0.275505   3.0
3    5000        4  296624     296624    12226    0.274182   4.0
4    5000        6  178812     178812    10276    0.230451   5.0


Here is the recommendation for the userId: 243

   userID  level_0   index  ProductID  Ratings  Percentage  Rank
0     243        1  429571     429571    16454    0.369000   1.0
1     243        2  327307     327307    14172    0.317823   2.0
2     243        3  102803     102803    12285    0.275505   3.0
3     243        4  296624     296624    12226    0.274182   4.0
4     243        6  178812     178812    10276    0.230451   5.0


Here is the recommendation for the userId: 5



C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


   userID  level_0   index  ProductID  Ratings  Percentage  Rank
0       5        1  429571     429571    16454    0.369000   1.0
1       5        2  327307     327307    14172    0.317823   2.0
2       5        3  102803     102803    12285    0.275505   3.0
3       5        4  296624     296624    12226    0.274182   4.0
4       5        6  178812     178812    10276    0.230451   5.0


Here is the recommendation for the userId: 34532

   userID  level_0   index  ProductID  Ratings  Percentage  Rank
0   34532        1  429571     429571    16454    0.369000   1.0
1   34532        2  327307     327307    14172    0.317823   2.0
2   34532        3  102803     102803    12285    0.275505   3.0
3   34532        4  296624     296624    12226    0.274182   4.0
4   34532        6  178812     178812    10276    0.230451   5.0


Here is the recommendation for the userId: 99999

   userID  level_0   index  ProductID  Ratings  Percentage  Rank
0   99999        1  429571     429571    16454    0

As it can be seen, the recommendation items for differnet user based on popularity are same.

# Item-Based recommender model

In [26]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.externals import joblib
import scipy.sparse
from scipy.sparse import csr_matrix

In [27]:
df3_items.head()

,ProductID,Ratings,Percentage
37,37,1051,0.023570
52,52,250,0.005607
56,56,139,0.003117
58,58,171,0.003835
59,59,484,0.010854


In [28]:
#item_index = df1_le.index[df3_items['ProductID']].tolist()
item_index = df1_le.index[df3_items['ProductID']].tolist()

In [29]:
len(item_index)

13077

In [30]:
ind = item_index[0:1000]
min(ind)

37

In [31]:
df1_le['UserId'] = le.fit_transform(df1_le['UserId'])

In [33]:
for_pivot = df1.iloc[df1_le['UserId'][ind[:]], :]

In [34]:
for_pivot.head()

,UserId,ProductID,Ratings
735675,1451728,44718,5.0
3740147,2075690,202447,2.0
1260191,3995230,73389,5.0
1270730,1048069,74091,4.0
3647038,2176262,198192,5.0


In [35]:
X2 = for_pivot.head(1000)

In [36]:
X2.drop_duplicates(['UserId', 'ProductID', 'Ratings'], keep='first', inplace=True)

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [37]:
X2.shape

(997, 3)

In [39]:
#pivot_df = for_pivot.pivot(index = 'UserId', columns ='ProductID', values = 'Ratings').fillna(0)
pivot_df = X2.pivot(index = 'UserId', columns ='ProductID', values = 'Ratings').fillna(0)
pivot_df.shape

(996, 939)

In [40]:
pivot_df['user_index'] = np.arange(0, pivot_df.shape[0], 1)
pivot_df.head()

ProductID,1466,1553,1745,2247,2757,2836,2874,2881,2966,3143,...,228022,228124,228180,228611,228920,229924,230393,230440,230455,user_index
UserId,,,,,,,,,,,,,,,,,,,,,
2953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
22639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
28636,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
34166,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
53174,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4


In [41]:
pivot_df.set_index(['user_index'], inplace=True)

# Actual ratings given by users
pivot_df.head()

ProductID,1466,1553,1745,2247,2757,2836,2874,2881,2966,3143,...,227922,228022,228124,228180,228611,228920,229924,230393,230440,230455
user_index,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# SVD Method 

In [42]:
from scipy.sparse.linalg import svds

In [43]:
U, sigma, vt = svds (pivot_df, k=10)

In [44]:
sigma = np.diag(sigma)

In [45]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), vt) 

preds_df = pd.DataFrame(all_user_predicted_ratings, columns = pivot_df.columns)
preds_df.head()

ProductID,1466,1553,1745,2247,2757,2836,2874,2881,2966,3143,...,227922,228022,228124,228180,228611,228920,229924,230393,230440,230455
0,-1.449251e-18,-1.094247e-20,2.120628e-17,2.201741e-17,-7.759219e-19,1.430265e-18,-7.550072e-18,1.898299e-18,-1.185864e-17,1.262904e-18,...,-1.956367e-18,6.075841e-18,-3.378260e-18,2.830524e-17,8.575305e-20,2.166735e-19,-6.502353e-19,2.088731e-18,-1.871772e-18,-1.121896e-17
1,2.200148e-34,5.438668e-37,-1.304443e-34,-1.748840e-34,-1.269254e-34,2.901339e-34,-9.320829e-34,-1.709204e-34,3.926693e-34,-1.742540e-34,...,1.066100e-34,-3.389745e-34,2.360720e-34,3.521205e-34,-3.402414e-35,3.798258e-34,2.102270e-34,-3.723896e-34,2.244981e-34,5.204938e-34
2,3.397046e-34,8.357818e-37,-1.077963e-34,-1.864380e-34,-2.258540e-34,5.094377e-34,-1.612027e-33,-2.706405e-34,7.966769e-34,-2.805989e-34,...,1.652613e-34,-5.234483e-34,3.518057e-34,7.710637e-34,-5.227722e-35,5.824638e-34,3.358813e-34,-5.842213e-34,4.027452e-34,8.047818e-34
3,4.326320e-33,9.668354e-36,1.387762e-33,1.324623e-33,-3.212600e-34,5.487872e-34,-2.418171e-33,-3.128714e-33,6.529586e-34,-2.375929e-33,...,1.992401e-33,-6.398604e-33,4.870520e-33,5.532564e-33,-6.229702e-34,8.270507e-33,3.547351e-33,-7.140627e-33,3.877423e-34,9.257453e-33
4,1.101554e-34,3.309987e-37,-1.808757e-33,-1.745116e-33,-9.571733e-35,2.084378e-34,-6.066866e-34,-1.110965e-34,9.758278e-34,-1.170607e-34,...,6.749779e-35,-2.147430e-34,1.335973e-34,4.033684e-34,-1.691821e-35,2.212416e-34,1.328451e-34,-2.443116e-34,2.020953e-34,3.394809e-34


In [46]:
# Recommend the items with the highest predicted ratings

def recommend_items(userID, pivot_df, preds_df, num_recommendations):
      
    user_idx = userID-1 # index starts at 0
    
    # Get and sort the user's ratings
    sorted_user_ratings = pivot_df.iloc[user_idx].sort_values(ascending=False)
    #sorted_user_ratings
    sorted_user_predictions = preds_df.iloc[user_idx].sort_values(ascending=False)
    #sorted_user_predictions

    temp = pd.concat([sorted_user_ratings, sorted_user_predictions], axis=1)
    temp.index.name = 'Recommended Items'
    temp.columns = ['user_ratings', 'user_predictions']
    
    temp = temp.loc[temp.user_ratings == 0]   
    temp = temp.sort_values('user_predictions', ascending=False)
    print('\nBelow are the recommended items for user(user_id = {}):\n'.format(userID))
    print(temp.head(num_recommendations))

In [47]:
userID = 2
num_recommendations = 5
recommend_items(userID, pivot_df, preds_df, num_recommendations)


Below are the recommended items for user(user_id = 2):

                   user_ratings  user_predictions
Recommended Items                                
171416                      0.0      1.135771e-17
179957                      0.0      2.337703e-18
201700                      0.0      2.202676e-18
107611                      0.0      1.960850e-18
96682                       0.0      9.489658e-19


C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


In [48]:
userID = 64
num_recommendations = 5
recommend_items(userID, pivot_df, preds_df, num_recommendations)


Below are the recommended items for user(user_id = 64):

                   user_ratings  user_predictions
Recommended Items                                
171416                      0.0      2.156946e-17
179957                      0.0      5.383444e-18
107611                      0.0      5.220438e-18
204698                      0.0      2.669305e-18
96682                       0.0      2.430892e-18


C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


In [49]:
userID = 345
num_recommendations = 5
recommend_items(userID, pivot_df, preds_df, num_recommendations)


Below are the recommended items for user(user_id = 345):

                   user_ratings  user_predictions
Recommended Items                                
179957                      0.0      9.327168e-18
201700                      0.0      6.279995e-18
107611                      0.0      5.514358e-18
187961                      0.0      4.545596e-18
167668                      0.0      4.255435e-18


C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


In [50]:
userID = 595
num_recommendations = 5
recommend_items(userID, pivot_df, preds_df, num_recommendations)


Below are the recommended items for user(user_id = 595):

                   user_ratings  user_predictions
Recommended Items                                
186025                      0.0      3.988196e-17
201700                      0.0      1.319304e-17
163556                      0.0      1.092458e-17
110803                      0.0      7.674971e-18
187961                      0.0      7.545668e-18


C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


Evaluation of Model-based Collaborative Filtering (SVD)

In [51]:
final_ratings_matrix = X2.pivot(index = 'UserId', columns ='ProductID', values = 'Ratings').fillna(0)

In [52]:
final_ratings_matrix.head()

ProductID,1466,1553,1745,2247,2757,2836,2874,2881,2966,3143,...,227922,228022,228124,228180,228611,228920,229924,230393,230440,230455
UserId,,,,,,,,,,,,,,,,,,,,,
2953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28636,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34166,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53174,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
final_ratings_matrix.shape

(996, 939)

In [54]:
final_ratings_matrix.mean().head()

ProductID
1466    0.005020
1553    0.005020
1745    0.002008
2247    0.002008
2757    0.004016
dtype: float64

In [55]:
preds_df.head()

ProductID,1466,1553,1745,2247,2757,2836,2874,2881,2966,3143,...,227922,228022,228124,228180,228611,228920,229924,230393,230440,230455
0,-1.449251e-18,-1.094247e-20,2.120628e-17,2.201741e-17,-7.759219e-19,1.430265e-18,-7.550072e-18,1.898299e-18,-1.185864e-17,1.262904e-18,...,-1.956367e-18,6.075841e-18,-3.378260e-18,2.830524e-17,8.575305e-20,2.166735e-19,-6.502353e-19,2.088731e-18,-1.871772e-18,-1.121896e-17
1,2.200148e-34,5.438668e-37,-1.304443e-34,-1.748840e-34,-1.269254e-34,2.901339e-34,-9.320829e-34,-1.709204e-34,3.926693e-34,-1.742540e-34,...,1.066100e-34,-3.389745e-34,2.360720e-34,3.521205e-34,-3.402414e-35,3.798258e-34,2.102270e-34,-3.723896e-34,2.244981e-34,5.204938e-34
2,3.397046e-34,8.357818e-37,-1.077963e-34,-1.864380e-34,-2.258540e-34,5.094377e-34,-1.612027e-33,-2.706405e-34,7.966769e-34,-2.805989e-34,...,1.652613e-34,-5.234483e-34,3.518057e-34,7.710637e-34,-5.227722e-35,5.824638e-34,3.358813e-34,-5.842213e-34,4.027452e-34,8.047818e-34
3,4.326320e-33,9.668354e-36,1.387762e-33,1.324623e-33,-3.212600e-34,5.487872e-34,-2.418171e-33,-3.128714e-33,6.529586e-34,-2.375929e-33,...,1.992401e-33,-6.398604e-33,4.870520e-33,5.532564e-33,-6.229702e-34,8.270507e-33,3.547351e-33,-7.140627e-33,3.877423e-34,9.257453e-33
4,1.101554e-34,3.309987e-37,-1.808757e-33,-1.745116e-33,-9.571733e-35,2.084378e-34,-6.066866e-34,-1.110965e-34,9.758278e-34,-1.170607e-34,...,6.749779e-35,-2.147430e-34,1.335973e-34,4.033684e-34,-1.691821e-35,2.212416e-34,1.328451e-34,-2.443116e-34,2.020953e-34,3.394809e-34


In [56]:
preds_df.mean().head()

ProductID
1466   -3.888982e-19
1553   -9.246905e-22
1745   -1.132191e-18
2247   -1.238178e-18
2757    1.129985e-19
dtype: float64

In [57]:
rmse_df = pd.concat([final_ratings_matrix.mean(), preds_df.mean()], axis=1)
rmse_df.columns = ['Avg_actual_ratings', 'Avg_predicted_ratings']
print(rmse_df.shape)
rmse_df['item_index'] = np.arange(0, rmse_df.shape[0], 1)
rmse_df.head()

(939, 2)


,Avg_actual_ratings,Avg_predicted_ratings,item_index
ProductID,,,
1466,0.005020,-3.888982e-19,0
1553,0.005020,-9.246905e-22,1
1745,0.002008,-1.132191e-18,2
2247,0.002008,-1.238178e-18,3
2757,0.004016,1.129985e-19,4


In [58]:
RMSE = round((((rmse_df.Avg_actual_ratings - rmse_df.Avg_predicted_ratings) ** 2).mean() ** 0.5), 5)
print('\nRMSE of SVD Model = {} \n'.format(RMSE))


RMSE of SVD Model = 0.00454 



The RMSE error is less, hence, the model for item based recommendation is good for the present dataset.

Summary: 
On given large dataset, first I made a subset of data for calculation of popularity based model. Please note that, in the popularity based model, the recommended items for every user would be same. Nect, I built a model for item based similarity. It is advicable to recommendation system on build item based  similarity rather than user based similarity as the mber of number of items in any dataset is limited while building user based similarity model becomes computationally intense.